# Compare Enzyme assay performed manually and using liquid handler #
### Notebook to generate Fig. 5 ###

### Import Necessary Modules ###

In [1]:
import sys
import colorlover as cl
from plotly import tools
import plotly.graph_objs as go
import chart_studio.plotly as py
import os
from copy import deepcopy
try:
    _ = __IPYTHON__
except NameError:
    from plotly.offline import plot
else:
    if 'ipykernel' in sys.modules:
        from plotly.offline import init_notebook_mode
        from plotly.offline import iplot as plot
        from IPython.display import HTML
        HTML("""
             <script>
              var waitForPlotly = setInterval( function() {
              if( typeof(window.Plotly) !== "undefined" ){
              MathJax.Hub.Config({ SVG: { font: "STIX-Web" }, displayAlign: "center" });
              MathJax.Hub.Queue(["setRenderer", MathJax.Hub, "SVG"]);
              clearInterval(waitForPlotly);}}, 250 );
            </script>
            """
        )
        init_notebook_mode(connected=True)
    elif 'IPython' in sys.modules:
        from plotly.offline import plot
    else:
        warn('Unknown ipython configuration')
        from plotly.offline import plot

import numpy as np
import pandas as pd
from lmfit import Model
import plotly.io as pio
from openpyxl import load_workbook
from scipy.optimize import curve_fit

pio.templates.default = "none"


C:\Users\User\anaconda3\envs\impact\lib\site-packages\IPython\html.py:14: ShimWarning:

The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.



# Define Curve Fit Function (Michaelis-Menten) #

In [2]:
def MicMenten(C, V_max, K_m):
    
    return V_max * C/(K_m + C)

# Preprocess Data #

In [3]:
#Data from automated assay
data_dict_liha = {'Rep1': [0, 0.09702045, 0.224911, 0.432182, 0.6835532, 0.9569744, 0.6835532, 1.014305],
                  'Rep2': [0, 0.07938037, 0.1852209, 0.3924918, 0.5733026, 0.7805736, 0.868774, 1.111325],
                  'Rep3': [0, 0.1278906, 0.1764008, 0.3616217, 0.5644826, 0.6703231, 0.7585235, 0.7011933]}

conc_list = [0, 0.05, 0.1, 0.25, 0.5, 1, 2, 5]

liha_df = pd.DataFrame(data=data_dict_liha)
liha_df['Mean'] = liha_df.mean(axis=1)
liha_df['Std'] = liha_df.std(axis=1)
liha_df['Concentrations'] = conc_list
liha_df = liha_df.set_index('Concentrations')

#Data from manual assay
data_dict_manual = {'Rep1': [0, 0.11025051, 0.229321061, 0.476282205, 0.635042939, 0.780573613, 0.882004083, 0.899644164],
                    'Rep2': [0, 0.097020449, 0.216091, 0.436592021, 0.604172797, 0.780573613, 0.793803674, 0.899644164],
                     'Rep3': [0, 0.145530674, 0.260191204, 0.485102245, 0.370441715, 0.820263797, 0.899644164, 0.952564409]}

manual_df = pd.DataFrame(data=data_dict_manual)
manual_df['Mean'] = manual_df.mean(axis=1)
manual_df['Std'] = manual_df.std(axis=1)
manual_df['Concentrations'] = conc_list
manual_df = manual_df.set_index('Concentrations')



# Perform Curve Fit #

In [4]:
# Change initial guesses here
V_max_init = .917
K_m_init = .245


params_dict_liha = {'average':{},
                         'stdev':{}}

params,cov = curve_fit(f=MicMenten, xdata=list(liha_df.index), ydata=liha_df['Mean'].values,
                       p0=[V_max_init, K_m_init],
                      maxfev=100000)
params_std = np.sqrt(np.diag(cov))

params_dict_liha['average']['V_max'],\
params_dict_liha['average']['K_m'] = params

params_dict_liha['stdev']['V_max'],\
params_dict_liha['stdev']['K_m']  = params_std



# Change initial guesses here
V_max_init = .917
K_m_init = .245

params_dict_manual = {'average':{},
                         'stdev':{}}

params,cov = curve_fit(f=MicMenten, xdata=list(manual_df.index), ydata=manual_df['Mean'].values,
                       p0=[V_max_init, K_m_init],
                      maxfev=100000)
params_std = np.sqrt(np.diag(cov))

params_dict_manual['average']['V_max'],\
params_dict_manual['average']['K_m'] = params

params_dict_manual['stdev']['V_max'],\
params_dict_manual['stdev']['K_m']  = params_std



# Plot fit and print fit parameters #

In [8]:
tracelist = []
colors = ['rgb(54, 146, 68)',
          'rgb(217, 72, 42)',
          'rgb(33, 100, 175)',
          'rgb(146, 54, 132)',
          'rgb(175, 127, 91)']
fillcolors = ['rgba(54, 146, 68, 0.25)',
              'rgba(217, 72, 42, 0.25)',
              'rgba(33, 100, 175, 0.25)',
              'rgba(146, 54, 132, 0.25)',
             'rgba(175, 127, 91, 0.25)']
    

fit_data = MicMenten(np.arange(0,conc_list[-1],0.01), params_dict_liha['average']['V_max'],
                                            params_dict_liha['average']['K_m'])

fit_plus = MicMenten(np.arange(0,conc_list[-1],0.01), params_dict_liha['average']['V_max']+params_dict_liha['stdev']['V_max'],
                                            params_dict_liha['average']['K_m']-params_dict_liha['stdev']['K_m'])

fit_minus = MicMenten(np.arange(0,conc_list[-1],0.01), params_dict_liha['average']['V_max']-params_dict_liha['stdev']['V_max'],
                                            params_dict_liha['average']['K_m']+params_dict_liha['stdev']['K_m'])
tracelist.append(go.Scatter(x=np.arange(0,conc_list[-1],0.01),y=fit_data, name='Michaelis-Menten Fit',line={'width': 2.5,'color': colors[0]}, mode='lines', showlegend=False)) 

fit_data = MicMenten(np.arange(0,conc_list[-1],0.01), params_dict_manual['average']['V_max'],
                                            params_dict_manual['average']['K_m'])

fit_plus = MicMenten(np.arange(0,conc_list[-1],0.01), params_dict_manual['average']['V_max']+params_dict_manual['stdev']['V_max'],
                                            params_dict_manual['average']['K_m']-params_dict_manual['stdev']['K_m'])

fit_minus = MicMenten(np.arange(0,conc_list[-1],0.01), params_dict_manual['average']['V_max']-params_dict_manual['stdev']['V_max'],
                                            params_dict_manual['average']['K_m']+params_dict_manual['stdev']['K_m'])
tracelist.append(go.Scatter(x=np.arange(0,conc_list[-1],0.01),y=fit_data, name='Michaelis-Menten Fit',line={'width': 2.5,'color': colors[1], 'dash':'dot'}, mode='lines'))


tracelist.append(go.Scatter(x=list(manual_df.index),y=manual_df.Mean.values, 
                            name='Experimental Data',  mode='markers', marker = dict(symbol='triangle-down', size=10, color='white', opacity=1, line=dict(width=1, color='white')),
                            error_y=dict(type='data',thickness=1.5, width=3,
                                         array=manual_df.Std.values,
                                         visible=True, color=colors[1]), showlegend=False))

tracelist.append(go.Scatter(x=list(liha_df.index),y=liha_df.Mean.values, 
                            name='Experimental Data',  mode='markers', marker = dict(symbol='triangle-up', size=.1, color='white', opacity=0.75, line=dict(width=1, color='white')),
                            error_y=dict(type='data',thickness=1.5, width=3, 
                                         array=liha_df.Std.values,
                                         visible=True, color=colors[0]),showlegend=False))

tracelist.append(go.Scatter(x=list(manual_df.index),y=manual_df.Mean.values, 
                            name='Experimental Data',  mode='markers', marker = dict(symbol='circle', size=10, color=colors[1], opacity=1, line=dict(width=1, color='black')), showlegend=False))

tracelist.append(go.Scatter(x=list(liha_df.index),y=liha_df.Mean.values, 
                            name='Experimental Data',  mode='markers', marker = dict(symbol='triangle-up', size=10, color=colors[0], opacity=0.75, line=dict(width=1, color='black'))))




layout = go.Layout(height=450, width=550, legend_x= 1.05, legend_font=dict(family='Myriad Pro', size=16, color='black'),
                   xaxis=dict(title='Substrate Concentration (mM)',
                              title_standoff=0.5,
                              titlefont=dict(family='Myriad Pro', size=16, color='black'),
                              showline=True, linewidth=1, linecolor='black', mirror=True, side='bottom',
                              ticks='outside', ticklen=4, tickangle=0, nticks=8,
                              tickfont=dict(size=16, family='Myriad Pro', color='black'), tickcolor='black',
                              showgrid=True,zeroline=False,range=(-0.1,5.1)),
                  
                  yaxis=dict(title='YqjM Activity (U/mg))',
                            titlefont=dict(family='Myriad Pro', size=16, color='black'),

                              anchor='x', side='left', showgrid=True, zeroline=False,
                            tickfont=dict(family='Myriad Pro',size=16, color='black'), tickcolor='black', 

                              range=(-0.05, 1.2),showline=True, linewidth=1, linecolor='black', mirror=True,
                              ticks='outside', ticklen=4, tickangle=0))



fig = go.Figure(data=tracelist, layout=layout)
plot(fig)
display(liha_df[['Mean','Std']])

print('Curve fit parameters For Michaelis-Menten equation with Hill coefficient:\n')
print('y = V_max * C / (K_M + C)\n')

print("Manual")
print('V_max = ', params_dict_manual['average']['V_max'], '\u00B1 ', params_dict_manual['stdev']['V_max'])
print('K_D = ', params_dict_manual['average']['K_m'], '\u00B1 ', params_dict_manual['stdev']['K_m'])

print("\n\nAuotmated")
print('V_max = ', params_dict_liha['average']['V_max'], '\u00B1 ', params_dict_liha['stdev']['V_max'])
print('K_D = ', params_dict_liha['average']['K_m'], '\u00B1 ', params_dict_liha['stdev']['K_m'])
#pio.write_image(fig,"Figures/fig4_a.svg",format='svg')



,Mean,Std
Concentrations,,
0.00,0.000000,0.000000
0.05,0.101430,0.020048
0.10,0.195511,0.021099
0.25,0.395432,0.028881
0.50,0.607113,0.054171
1.00,0.802624,0.118059
2.00,0.770284,0.076072
5.00,0.942274,0.175011


Curve fit parameters For Michaelis-Menten equation with Hill coefficient:

y = V_max * C / (K_M + C)

Manual
V_max =  0.9919035091309237 ±  0.03433513887498563
K_D =  0.3251014621568612 ±  0.04074547056542252


Auotmated
V_max =  0.993637108730265 ±  0.04435253542542222
K_D =  0.35290901829152727 ±  0.05586180220274018


### Salt and pH concentration gradient activity Data ###

In [9]:
pH = [2.2, 3, 4, 5, 6, 7, 8]
pH_data_avg = [-0.0396901836666667,-0.047040218,0.151410700666667,1.45824675,1.19658553866667,1.13925527333333,1.262735845]
pH_data_errors = [0.102921474288931,0.00207890383618339,0.00415780672955773,0.0340331182365778,0.0245099113554667,0.0198314760340331,0.0260485868506243]

### YqjM activity over different pH levels###

In [11]:
colors = ['rgb(54, 146, 68)', 'rgb(54, 146, 68)', 'rgb(54, 146, 68)']


trace_list = []

trace_list.append(go.Bar(x=pH,
                         y=pH_data_avg, 
                         error_y=dict(type='data',
                                      array=pH_data_errors, 
                                      visible=True, width=3, thickness=1.5, color='black'),
                         marker=dict(color=colors[2], line=dict(width=0, color='black')),
                         name='Activity',showlegend=False))

layout = go.Layout(height=400, width=475, showlegend=True, bargroupgap=0.1, bargap=0.5,
                   legend_orientation='v',legend_y=-0, legend_font=dict(family='Arial', color='black', size=14),
                   xaxis2=dict(visible=False, anchor='y', overlaying='x', range=(0,4)),

                    xaxis=dict(title_text='Salt Concentration', 
                               title_font=dict(family='Myriad Pro',size=16,color='black'),
                               showline=True, linewidth=1, linecolor='black', mirror=True, showgrid=False,type='category',
                               ticks='outside', ticklen=4, tickfont=dict(size=16, family='Myriad Pro', color='black'),
                               tickangle=-0, tickcolor='black'),

                   yaxis=dict(title='Activity<br>(U/mg)',
                              titlefont=dict(family='Myriad Pro', size=16, color='black'), 
                              showline=True, linewidth=1, linecolor='black', mirror=True, showgrid=True,
                              ticks='outside', ticklen=4, tickfont=dict(family='Myriad Pro',size=16, color='black'),
                              tickangle=0, nticks=5, tickcolor='black',range=(-0.15,1.6)))

fig = go.Figure(data=trace_list,layout=layout)
plot(fig)
#pio.write_image(fig,"Figures/fig4_c.svg",format='svg')
